In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

In [4]:
data=pd.read_csv("/content/drive/MyDrive/outlier/윤석열.csv")
data.head()


,tweet,date,retweets_count,link
0,윤석열 나경원 김태호 인성 덜 된 자들의 헤쳐모여는 파국으로 치닫을 것 상식과 공정...,2021-11-22,0,https://twitter.com/Mesmerizingway/status/1462...
1,티비조선윤석열티비조선윤석열티비조선윤석열티비조선윤석열티비조선윤석열티비조선윤석열티비조선...,2021-11-22,0,https://twitter.com/songsun21/status/146277414...
2,그걸 윤석열 지지로 읽으니까 무식하다는 소리를 하지 그리고 니네들 한심한 거 맞잖아,2021-11-22,0,https://twitter.com/coervos/status/14627741373...
3,민주당은 국힘당 대변인 김병민을 허위사실 유포로 즉각 고발해야 가짜 영수증으로 윤석...,2021-11-22,0,https://twitter.com/dalsoo7/status/14627741300...
4,날씨 차진다 이럴 때는 따끈한 오뎅에 떡볶이가 최고죠 건 그거고 티비조선에서 윤석열...,2021-11-22,0,https://twitter.com/RGbVaZ8bF5oodRN/status/146...


In [30]:
range(len(data["tweet"]))#27943데이터

range(0, 27943)

# 정수 인코딩과 단어 집합 만들기

In [31]:
!pip install konlpy
from konlpy.tag import Kkma  
kkma = Kkma()
tokenized_doc=[]
for n in range(len(data["tweet"])):
    text = data["tweet"][n]
    tokenized_doc.append(list(kkma.nouns(str(text))))
print(tokenized_doc)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [32]:
print(tokenized_doc[2])

['그것', '윤', '지지', '무식', '무식하다', '하다', '소리', '거']


In [35]:
kkma.nouns(data["tweet"][2])

['그것', '윤', '지지', '무식', '무식하다', '하다', '소리', '거']

이제 각 단어에 정수 인코딩을 하는 동시에, 각 뉴스에서의 단어의 빈도수를 기록. 여기서는 각 단어를 (word_id, word_frequency)의 형태로 바꾸고자 합니다. word_id는 단어가 정수 인코딩된 값이고, word_frequency는 해당 뉴스에서의 해당 단어의 빈도수를 의미합니다. 이는 gensim의 corpora.Dictionary()를 사용하여 손쉽게 구할 수 있습니다. 전체 뉴스에 대해서 정수 인코딩을 수행하고, 세번째 댓글을 추출합니다.

In [36]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[2])#세번째 댓글 출력

[(10, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)]


In [37]:
tokenized_doc[2]

['그것', '윤', '지지', '무식', '무식하다', '하다', '소리', '거']

In [38]:
print(corpus[2])#3번째 댓글아이디 출력

[(10, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1)]


총 학습된 단어의 개수를 확인

In [39]:
len(dictionary) #385505개 단어가 학습

38550

#  LDA 모델을 훈련

각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도를 보여줍니다. 또한 맨 앞에 있는 토픽 번호는 0부터 시작하므로 총 20개의 토픽은 0부터 19까지의 번호가 할당되어져 있습니다. passes는 알고리즘의 동작 횟수를 말하는데, 알고리즘이 결정하는 토픽의 값이 적절히 수렴할 수 있도록 충분히 적당한 횟수를 정해주면 됩니다. 여기서는 총 15회를 수행하였습니다. 여기서는 num_words=4로 총 4개의 단어만 출력하도록 하였습니다. 만약 10개의 단어를 출력하고 싶다면 아래의 코드를 수행하면 됩니다.

### 토픽번호별 키워드중요도

In [50]:
import gensim
NUM_TOPICS = 20 #20개의 토픽(카테고리), k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=7)
for topic in topics:
    print(topic)

(0, '0.048*"윤" + 0.033*"열" + 0.017*"이재명" + 0.012*"대통령" + 0.012*"안" + 0.011*"철수" + 0.008*"안철수"')
(1, '0.118*"다음" + 0.118*"뉴스" + 0.054*"윤" + 0.034*"다음뉴스" + 0.018*"열" + 0.017*"8" + 0.015*"0"')
(2, '0.043*"조사" + 0.037*"여론" + 0.027*"여론조사" + 0.021*"지지율" + 0.020*"격차" + 0.019*"이재명" + 0.016*"윤"')
(3, '0.046*"김건희" + 0.040*"김" + 0.038*"건희" + 0.031*"윤" + 0.028*"조작" + 0.025*"주가" + 0.021*"장모"')
(4, '0.077*"윤" + 0.066*"열" + 0.052*"이재명" + 0.037*"출처" + 0.033*"뉴스" + 0.024*"4" + 0.024*"1"')
(5, '0.046*"종" + 0.042*"윤" + 0.042*"부세" + 0.041*"종부세" + 0.019*"1" + 0.017*"열" + 0.015*"부자"')
(6, '0.024*"박" + 0.020*"년" + 0.020*"윤" + 0.017*"검찰" + 0.013*"권오수" + 0.011*"기소" + 0.010*"영수"')
(7, '0.034*"윤" + 0.022*"목포" + 0.021*"열" + 0.020*"논란" + 0.014*"폭탄주" + 0.013*"만찬" + 0.011*"위반"')
(8, '0.063*"표" + 0.055*"홍" + 0.045*"윤" + 0.037*"국민" + 0.029*"동" + 0.026*"당" + 0.026*"대장"')
(9, '0.043*"윤" + 0.027*"열" + 0.019*"사과" + 0.012*"광주" + 0.012*"조선" + 0.010*"36" + 0.010*"허"')
(10, '0.053*"윤" + 0.024*"열" + 0.019*"길" + 0.013*"거" + 0.0

In [51]:
print(ldamodel.print_topics())

[(0, '0.048*"윤" + 0.033*"열" + 0.017*"이재명" + 0.012*"대통령" + 0.012*"안" + 0.011*"철수" + 0.008*"안철수" + 0.008*"박근" + 0.007*"일" + 0.007*"수"'), (1, '0.118*"다음" + 0.118*"뉴스" + 0.054*"윤" + 0.034*"다음뉴스" + 0.018*"열" + 0.017*"8" + 0.015*"0" + 0.014*"1" + 0.013*"3" + 0.012*"9"'), (2, '0.043*"조사" + 0.037*"여론" + 0.027*"여론조사" + 0.021*"지지율" + 0.020*"격차" + 0.019*"이재명" + 0.016*"윤" + 0.014*"열" + 0.013*"양자" + 0.013*"45"'), (3, '0.046*"김건희" + 0.040*"김" + 0.038*"건희" + 0.031*"윤" + 0.028*"조작" + 0.025*"주가" + 0.021*"장모" + 0.020*"주가조작" + 0.019*"부인" + 0.016*"구속"'), (4, '0.077*"윤" + 0.066*"열" + 0.052*"이재명" + 0.037*"출처" + 0.033*"뉴스" + 0.024*"4" + 0.024*"1" + 0.024*"6" + 0.023*"3" + 0.023*"네이버"'), (5, '0.046*"종" + 0.042*"윤" + 0.042*"부세" + 0.041*"종부세" + 0.019*"1" + 0.017*"열" + 0.015*"부자" + 0.014*"부동산" + 0.014*"폭탄" + 0.011*"세금"'), (6, '0.024*"박" + 0.020*"년" + 0.020*"윤" + 0.017*"검찰" + 0.013*"권오수" + 0.011*"기소" + 0.010*"영수" + 0.010*"열" + 0.010*"4" + 0.010*"박영수"'), (7, '0.034*"윤" + 0.022*"목포" + 0.021*"열" + 0.020*"논란" + 0.014

# 문서 관점에서 topic과 document 알기(문서별 토픽 분포 보기)

토픽별 단어 분포는 위에서 확인하였음. 문서별 토픽 분포에 대해 확인을 위해 이미 훈련된 LDA모델인 ldamodel[]에 전체 데이터가 정수 인코딩 된 결과를 넣은 후 확인 가능. 

상위 5개의 문서에 대해서만 토픽분포 확인

In [52]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(3, 0.25302717), (7, 0.25270706), (10, 0.06613306), (11, 0.14127937), (19, 0.2451867)]
1 번째 문서의 topic 비율은 [(12, 0.28121677), (13, 0.5687832)]
2 번째 문서의 topic 비율은 [(19, 0.89444447)]
3 번째 문서의 topic 비율은 [(7, 0.63242215), (12, 0.09474277), (18, 0.041312285), (19, 0.20189314)]
4 번째 문서의 topic 비율은 [(4, 0.10213815), (9, 0.15248063), (10, 0.065371454), (13, 0.122473955), (16, 0.1808673), (17, 0.2934186), (19, 0.061583254)]


위의 출력 결과에서 (숫자, 확률)은 토픽 번호와 해당토픽이 해당문서에서 차지하는 분포도를 의미.예를 들어 2번째 문서의 토픽 비율에서 (5, 0.16109002)은 5번 토픽이 16%의 분포도를 가지는 것을 의미

좀 더 깔끔한 형태의 데이터프레임 형식으로 출력

In [53]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [54]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]#상위 10개 댓글별 비중이 높은 토픽은 무엇이고 얼만큼 차지하는지

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,3.0,0.2530,"[(3, 0.2530023), (7, 0.2527093), (10, 0.066136..."
1,1,13.0,0.5688,"[(12, 0.28123522), (13, 0.56876475)]"
2,2,19.0,0.8944,"[(19, 0.89444447)]"
3,3,7.0,0.6324,"[(7, 0.6324267), (12, 0.094749175), (18, 0.041..."
4,4,17.0,0.2934,"[(4, 0.10213033), (9, 0.15247975), (10, 0.0653..."
5,5,13.0,0.4677,"[(0, 0.052061476), (12, 0.0696473), (13, 0.467..."
6,6,13.0,0.5804,"[(4, 0.30169094), (10, 0.055411104), (13, 0.58..."
7,7,0.0,0.3814,"[(0, 0.3814288), (1, 0.35827836), (13, 0.21555..."
8,8,11.0,0.3947,"[(11, 0.39466307), (12, 0.24045952), (15, 0.30..."
9,9,8.0,0.8776,"[(8, 0.8776482), (18, 0.09588127)]"


# 단어 관점에서topic과 documents를 알기


In [62]:
keyword="세금"
id_list=[]
for i in range(len(data)):
  if keyword in tokenized_doc[i]:
    print("해당 단어가 들어간 댓글:",i,"번 댓글",":",data['tweet'][i])
    for y in range(len(tokenized_doc[i])):
      if tokenized_doc[i][y]==keyword:
        key_id=corpus[i][y]
        print("해당단어의 아이디는",key_id)
        id_list.append(key_id[0])

해당 단어가 들어간 댓글: 670 번 댓글 : 참여연대 16억 주택 소유자 종부세 20만원 수준 세금폭탄인가 SfTDlkVlx9 국민의힘 윤석열 후보에 공개질의서 발송 고령층 종부세 대상자도 연 소득 1억 넘어 
해당단어의 아이디는 (1064, 1)
해당 단어가 들어간 댓글: 921 번 댓글 :  대상자만 100만명 육박 역대급 종부세 커지는 조세저항 조세저항 저항한다고 세무사 변호사에 줄 돈으로 그냥 세금내라 그리고 세금 적게 걷어가는 나라로 이민가 가보면 알겠지 내는 세금만큼 혜택받는 나라가 있는지 pn0rjFNAHO
해당단어의 아이디는 (3878, 1)
해당 단어가 들어간 댓글: 1060 번 댓글 : 종부세 폭탄 종부세는 국민 상위 1 4 부자들이 부담하는 세금인데 이것을 국민 모두가 부담하는 세금으로 오인하겠금 보도하고 윤석열은 이런 세금도 깍아주겠다고 홍보하고 있습니다 세계 각국이 코로나로 증세로 가고 특히 부자들에 대한 세금이 대폭 오르고 있는데 깍는다고 미친놈 욕나오네 u0DeKWZyqr
해당단어의 아이디는 (276, 1)
해당 단어가 들어간 댓글: 1711 번 댓글 :  lt 윤석열 gt 은 종부세 대상자로서 1가구1주택자이다 윤석열은 종부세 세금이 아까워서 종부세 폐지 공약 을 발표했나 보다 윤서결 종부세
해당단어의 아이디는 (3198, 1)
해당 단어가 들어간 댓글: 1816 번 댓글 : 윤석열 대통령 골때리지만 김건희 영부인은 정말 못받아주겠다 사기범죄에 학력위조 학위위조 논문위조 한사람을 우리세금으로 영부인 대우를 해야해 
해당단어의 아이디는 (5983, 1)
해당 단어가 들어간 댓글: 2120 번 댓글 :  ojg4008 거봐 종부세 없애면 근로소득자면 세금 는다 했지 이런대도 윤석열뽑을래 난 이거보고 이재명을 뽑기로함
해당단어의 아이디는 (934, 1)
해당 단어가 들어간 댓글: 2904 번 댓글 : 윤석열 종부세 재검토 에 참여연대가 16억 아파트 따져봤다 pFxTXXyBfO 20만원 수준 1주택 고령자 최대공제 시 4만원 세금폭탄 

In [64]:
# 세금에 관련된 토픽(+토픽에 영향주는 키워드) 찾기
term_topic_dist = ldamodel.get_term_topics( 27466, minimum_probability=0)
sorted_term_topic = sorted(term_topic_dist, key=lambda x:x[1], reverse=True)
sorted_term_topic

[(19, 7.747001e-06)]

In [65]:
topics[19]

(19,
 '0.064*"윤" + 0.045*"열" + 0.032*"후보" + 0.026*"이재명" + 0.024*"힘" + 0.020*"국민" + 0.020*"대선"')

In [66]:
for n in range(len(id_list)):
  term_topic_dist = ldamodel.get_term_topics(id_list[n], minimum_probability=0)
  sorted_term_topic = sorted(term_topic_dist, key=lambda x:x[1], reverse=True)
  print("토픽중요도:",round(int(sorted_term_topic[0][1]*100),2),"%","토픽번호 및 해당키워드중요도:",topics[sorted_term_topic[0][0]])

토픽중요도: 1 % 토픽번호 및 해당키워드중요도: (5, '0.046*"종" + 0.042*"윤" + 0.042*"부세" + 0.041*"종부세" + 0.019*"1" + 0.017*"열" + 0.015*"부자"')
토픽중요도: 0 % 토픽번호 및 해당키워드중요도: (16, '0.034*"윤" + 0.016*"혜" + 0.015*"경" + 0.015*"개" + 0.015*"열" + 0.014*"질문" + 0.013*"사과"')
토픽중요도: 2 % 토픽번호 및 해당키워드중요도: (4, '0.077*"윤" + 0.066*"열" + 0.052*"이재명" + 0.037*"출처" + 0.033*"뉴스" + 0.024*"4" + 0.024*"1"')
토픽중요도: 0 % 토픽번호 및 해당키워드중요도: (18, '0.036*"문" + 0.027*"윤" + 0.027*"재인" + 0.022*"문재인" + 0.017*"기" + 0.017*"정부" + 0.016*"검찰"')
토픽중요도: 0 % 토픽번호 및 해당키워드중요도: (5, '0.046*"종" + 0.042*"윤" + 0.042*"부세" + 0.041*"종부세" + 0.019*"1" + 0.017*"열" + 0.015*"부자"')
토픽중요도: 0 % 토픽번호 및 해당키워드중요도: (16, '0.034*"윤" + 0.016*"혜" + 0.015*"경" + 0.015*"개" + 0.015*"열" + 0.014*"질문" + 0.013*"사과"')
토픽중요도: 0 % 토픽번호 및 해당키워드중요도: (5, '0.046*"종" + 0.042*"윤" + 0.042*"부세" + 0.041*"종부세" + 0.019*"1" + 0.017*"열" + 0.015*"부자"')
토픽중요도: 0 % 토픽번호 및 해당키워드중요도: (5, '0.046*"종" + 0.042*"윤" + 0.042*"부세" + 0.041*"종부세" + 0.019*"1" + 0.017*"열" + 0.015*"부자"')
토픽중요도: 0 % 토픽번호 및 해당키워드중요도: (5, 